# 03 - Modeling

Build models to predict "Percent_Bleached".
Start by loading your cleaned dataset, then build a pipeline for pre-processing the data (dealing with missing values, standardizing columns etc.) and training different predictive models.

Assess model quality by using the "Mean Absolute Error (MAE)" performance metric.

After you have successfully trained a model, predict new bleaching values for the blinded dataset.

In [ ]:
# Import for main performance metric (MAE)
from sklearn.metrics import mean_absolute_error

### Load The Blinded dataset

In [ ]:
import os
import pandas as pd

filepath_blinded = r"../data/coral_students_blinded.csv"
df_blinded = pd.read_csv(filepath_blinded)

df_blinded.head()